In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# 1. Nama file dataset yang akan digunakan
file_csv = "dataset_rumah_besar.csv" # Menggunakan dataset yang baru dibuat

# 2. Membaca Dataset dari CSV
print(f"Membaca dataset dari '{file_csv}'...")
try:
    df_loaded = pd.read_csv(file_csv)
except FileNotFoundError:
    print(f"Error: File '{file_csv}' tidak ditemukan. Pastikan file tersebut ada di direktori yang sama dengan script ini.")
    exit()

print("\nContoh 5 baris pertama dari dataset yang dimuat:")
print(df_loaded.head())

# Pisahkan fitur (X) dan label (y)
X = df_loaded.drop('layak_beli', axis=1)
y = df_loaded['layak_beli']

# Bagi data menjadi training dan testing set
# stratify=y digunakan untuk menjaga proporsi kelas pada data training dan testing,
# penting terutama jika dataset tidak seimbang.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

print(f"\nJumlah data training: {len(X_train)}")
print(f"Jumlah data testing: {len(X_test)}")
print(f"Proporsi kelas 'layak_beli' di training set:\n{y_train.value_counts(normalize=True)}")
print(f"Proporsi kelas 'layak_beli' di testing set:\n{y_test.value_counts(normalize=True)}")


# 3. Mendefinisikan fitur numerik dan kategorikal
# Pastikan nama kolom sesuai dengan yang ada di dataset_rumah_lebih_besar.csv
numeric_features = ['harga', 'luas', 'jumlah_kamar']
categorical_features = ['lokasi']

# Membuat preprocessor menggunakan ColumnTransformer
# StandardScaler untuk fitur numerik, OneHotEncoder untuk fitur kategorikal
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# 4. Membuat Pipeline untuk Model RandomForestClassifier
# Pipeline menggabungkan langkah preprocessing dan model classifier
pipeline_rf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# 5. Definisi Parameter Grid untuk GridSearchCV
# Ini adalah daftar parameter yang akan diuji oleh GridSearchCV
# Anda bisa menambahkan atau mengubah parameter dan nilainya sesuai kebutuhan
param_grid_rf = {
    'classifier__n_estimators': [50, 100, 150], # Jumlah pohon dalam forest
    'classifier__max_depth': [None, 10, 20],    # Kedalaman maksimum pohon
    'classifier__min_samples_split': [2, 5, 10], # Jumlah minimum sampel untuk membagi node
    'classifier__min_samples_leaf': [1, 2, 4]    # Jumlah minimum sampel pada leaf node
}

# 6. Melakukan GridSearchCV untuk Tuning Hyperparameter
# cv=5 berarti menggunakan 5-fold cross-validation
# scoring='accuracy' berarti model dievaluasi berdasarkan akurasi
# n_jobs=-1 menggunakan semua core CPU yang tersedia untuk mempercepat proses
print("\nMelakukan GridSearchCV untuk tuning hyperparameter RandomForestClassifier...")
grid_search_rf = GridSearchCV(pipeline_rf, param_grid_rf, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)
grid_search_rf.fit(X_train, y_train)

# Menampilkan parameter terbaik yang ditemukan
print("\nParameter terbaik yang ditemukan untuk RandomForestClassifier: ")
print(grid_search_rf.best_params_)

# Mendapatkan model terbaik dari hasil grid search
best_rf_model = grid_search_rf.best_estimator_

# 7. Evaluasi Model Terbaik pada Data Testing
print("\nMelakukan prediksi pada data testing dengan model RandomForestClassifier terbaik...")
y_pred_rf = best_rf_model.predict(X_test)

print("\nClassification Report (RandomForestClassifier):")
print(classification_report(y_test, y_pred_rf))

print("\nConfusion Matrix (RandomForestClassifier):")
print(confusion_matrix(y_test, y_pred_rf))

# 8. Contoh Prediksi pada Data Baru
# Buat DataFrame dengan data baru yang ingin diprediksi
# Pastikan kolomnya sama dengan yang digunakan saat training
data_rumah_baru = pd.DataFrame({
    'lokasi': ['Jakarta', 'Bandung', 'Surabaya', 'Medan', 'Semarang'],
    'harga': [1200, 2000, 800, 1450, 3000],
    'luas': [100, 150, 70, 120, 200],
    'jumlah_kamar': [3, 4, 2, 3, 5]
})

print("\nData rumah baru untuk diprediksi:")
print(data_rumah_baru)

prediksi_baru_rf = best_rf_model.predict(data_rumah_baru)
prediksi_proba_baru_rf = best_rf_model.predict_proba(data_rumah_baru) # Probabilitas prediksi

data_rumah_baru['prediksi_layak_beli'] = prediksi_baru_rf
data_rumah_baru['probabilitas_tidak_layak'] = prediksi_proba_baru_rf[:, 0]
data_rumah_baru['probabilitas_layak'] = prediksi_proba_baru_rf[:, 1]

print("\nHasil prediksi untuk rumah baru (0=Tidak Layak, 1=Layak Beli):")
print(data_rumah_baru[['lokasi', 'harga', 'luas', 'jumlah_kamar', 'prediksi_layak_beli', 'probabilitas_layak']])

# Menambahkan informasi mengenai kriteria kelayakan beli yang digunakan pada dataset
print("\nCatatan: Dataset 'dataset_rumah_besar.csv' dibuat dengan kriteria 'layak_beli':")
print("((harga <= 1500) & (jumlah_kamar >= 2))")

Membaca dataset dari 'dataset_rumah_besar.csv'...

Contoh 5 baris pertama dari dataset yang dimuat:
       lokasi  harga  luas  jumlah_kamar  layak_beli
0    Makassar   1939   113             4           0
1       Medan   1127   260             5           1
2   Palembang   3030   190             3           0
3  Yogyakarta   3831    67             2           0
4    Makassar   4751   255             4           0

Jumlah data training: 750
Jumlah data testing: 250
Proporsi kelas 'layak_beli' di training set:
layak_beli
0    0.797333
1    0.202667
Name: proportion, dtype: float64
Proporsi kelas 'layak_beli' di testing set:
layak_beli
0    0.8
1    0.2
Name: proportion, dtype: float64

Melakukan GridSearchCV untuk tuning hyperparameter RandomForestClassifier...
Fitting 5 folds for each of 81 candidates, totalling 405 fits

Parameter terbaik yang ditemukan untuk RandomForestClassifier: 
{'classifier__max_depth': None, 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2,

In [4]:
import streamlit as st
import pandas as pd
import joblib
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

# Judul Aplikasi
st.title("Aplikasi Prediksi Kelayakan Beli Rumah")
st.write("Aplikasi ini memprediksi apakah sebuah rumah 'layak beli' berdasarkan kriteria tertentu.")

# Muat model dan preprocessor menggunakan caching Streamlit
# Caching memastikan model hanya dimuat sekali saat aplikasi dimulai
@st.cache_resource
def load_model_and_preprocessor():
    try:
        model = joblib.load('best_rf_model.pkl')
        # Preprocessor juga perlu dimuat
        # Jika preprocessor memiliki bagian yang perlu data asli untuk fit (misalnya OHE dengan kategori),
        # cara terbaik adalah menyimpan pipeline secara utuh seperti di bawah.
        # Namun, jika Anda menyimpan preprocessor terpisah, pastikan ia siap transform.
        # Untuk contoh ini, asumsikan pipeline lengkap disimpan.
        # Jika Anda menyimpan preprocessor terpisah, pastikan muat dengan joblib.load('preprocessor.pkl')
        return model # Asumsi model adalah pipeline lengkap
    except FileNotFoundError:
        st.error("File model atau preprocessor tidak ditemukan. Pastikan 'best_rf_model.pkl' ada di direktori yang sama.")
        return None # Mengembalikan None jika file tidak ditemukan

pipeline_rf = load_model_and_preprocessor()

if pipeline_rf is not None:
    st.header("Masukkan Data Rumah Baru")

    # Input dari pengguna
    lokasi = st.selectbox("Lokasi", ['Jakarta', 'Bandung', 'Surabaya', 'Medan', 'Semarang', 'Lainnya']) # Sesuaikan dengan lokasi di dataset Anda
    harga = st.number_input("Harga (dalam ratusan juta)", min_value=1, value=1000)
    luas = st.number_input("Luas (dalam meter persegi)", min_value=1, value=100)
    jumlah_kamar = st.number_input("Jumlah Kamar", min_value=1, value=3)

    # Tombol Prediksi
    if st.button("Prediksi Kelayakan Beli"):
        # Buat DataFrame dari input pengguna
        data_baru = pd.DataFrame({
            'lokasi': [lokasi],
            'harga': [harga],
            'luas': [luas],
            'jumlah_kamar': [jumlah_kamar]
        })

        # Lakukan prediksi menggunakan pipeline
        # Pipeline sudah mencakup preprocessing dan prediksi model
        try:
            prediksi = pipeline_rf.predict(data_baru)
            prediksi_proba = pipeline_rf.predict_proba(data_baru)

            # Tampilkan hasil prediksi
            st.subheader("Hasil Prediksi")
            if prediksi[0] == 1:
                st.success(f"Rumah ini **Layak Beli**")
            else:
                st.error(f"Rumah ini **Tidak Layak Beli**")

            st.write(f"Probabilitas Layak Beli: **{prediksi_proba[0][1]:.2f}**")
            st.write(f"Probabilitas Tidak Layak Beli: **{prediksi_proba[0][0]:.2f}**")

            st.write("\nCatatan: Kriteria 'layak_beli' pada model ini didasarkan pada pola di dataset pelatihan.")

        except Exception as e:
            st.error(f"Terjadi kesalahan saat melakukan prediksi: {e}")

else:
    st.warning("Aplikasi tidak dapat berjalan karena file model tidak ditemukan.")


st.markdown("""
---
*Dibuat dengan Streamlit*
""")

2025-06-03 03:33:02.502 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-03 03:33:02.745 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-06-03 03:33:02.754 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-03 03:33:02.759 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-03 03:33:02.763 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-03 03:33:02.768 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-03 03:33:02.774 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-03 03:33:02.777 Thread 'MainThread': mi

DeltaGenerator()